### 라이브러리 불러오기

In [1]:
import os
import sys
import urllib.request
import datetime
import time
import json

### 기본 틀

In [ ]:
client_id = '0XTfDtoDevGZuhx3BAck'
client_secret = '2Nxn8f6OdL'


# [CODE 1]
def getRequestUrl(url):    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200: # 400
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
        else:
            return None
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

# [CODE 2]
def getNaverSearch(node, srcText, start, display):    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display) # .parse.quote는 이제 srcText를 인코딩 해 넣는다. 한글은 깨지기 때문에.
    
    url = base + node + parameters    
    responseDecode = getRequestUrl(url)   # [CODE 1] json 형식의 utf8로 인코딩된 문자열 형태다.
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode) # 응답 객체를 파이썬이 처리할 수 있는 json 형식으로 변환한다. 즉, json 파일을 딕셔너리로 변환한다. 왜냐면 나중에 items에 접근해야 하니깐 파이썬에서.

# [CODE 3]
def getPostData(post, jsonResult, cnt):    
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    
    pDate = datetime.datetime.strptime(post['pubDate'],  '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 
'org_link':org_link,   'link': org_link,   'pDate':pDate})
        

# [CODE 0]
def main():
    node = 'news'   # 크롤링 대상 노드 : 네이버 '뉴스' # 네이버 검색 API에서 검색할 대상 노드
    srcText = input('검색어를 입력하세요: ') # 사용자 입력으로 받은 검색어 저장
    cnt = 0 # 검색 결과 카운트
    jsonResult = [] # 검색 결과를 정리하여 저장할 리스트 객체.
    # json파일은 약간 딕셔너리 형태임. key - value
    jsonResponse = getNaverSearch(node, srcText, 1, 100)  # [CODE 2] 네이버 뉴스 검색에 대한 응답을 저장하는 객체.
    
    total = jsonResponse['total'] # 전체 검색 결과 개수
    # jsonFile: JSON파일에 저장할 데이터를 담은 객체.
    # total = 350   한 번에 items를 100개가. 101 200
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)): # display = 한 번에 가져온 결과 개수.  즉, display = len(items)이다.
        for post in jsonResponse['items']: # post = 응답받은 검색 결과 중 1개 저장한 객체 / items = 응답받은 검색 결과 전체. 내부 항목은 title, originallink,link,description, pubDate 등.
            cnt += 1                       # title = 기사 제목, originallink = 원래 언론사 사이트 주소, link = 네이버 뉴스 주소, description = 기사 요약, 본문 일부, pubDate = 기사 발행 시각
            getPostData(post, jsonResult, cnt)  # [CODE 3]   # getPostData = 검색 결과 1개를 저장함.     
        
        start = jsonResponse['start'] + jsonResponse['display'] # 이제 다음 items들을 가져와야 하니깐 이렇게 함.
        if start == 1001: break    # 네이버 뉴스는 1000개까지만 무료 제공됨
        jsonResponse = getNaverSearch(node, srcText, start, 100)  #[CODE 2]       

    print('전체 검색 : %d 건' %total)
    
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult,  indent = 4, sort_keys = True,  ensure_ascii = False) # 객체를 JSON 형식으로 변환.
                        
        outfile.write(jsonFile)
        
    print("가져온 데이터 : %d 건" %(cnt))
    print ('%s_naver_%s.json SAVED' % (srcText, node))
    
if __name__ == '__main__':
    main()


### 연습하기3

In [4]:
import json
import urllib.request
import datetime
import re
import pandas as pd

client_id = '0XTfDtoDevGZuhx3BAck'
client_secret = '2Nxn8f6OdL'

lst_subin = ["EXO", "존박", "수지", "sia","Jordin Sparks", "규현", "스텔라장", "뷔", "BTS", "박효신", "보이넥스트도어", "SEINY", "써니힐", "임영웅", "강수지", "빅뱅", "조용필"]
lst_seonggu = ["석화", "과메기", "대게", "방어","호떡", "붕어빵", "풀빵", "국밥", "국밥", "오뎅", "만두", "전골", "스테이크"]
lst_taehoon = ["서울", "강원도", "부산", "경주", "제주", "여수", "포항", "인천", "강릉"]

diction = dict()

# [CODE 1]
def getRequestUrl(url):    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200: # 400
            return response.read().decode('utf-8')
        else:
            return None
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

# [CODE 2]
def getNaverSearch(node, srcText, start, display):    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display) # .parse.quote는 이제 srcText를 인코딩 해 넣는다. 한글은 깨지기 때문에.
    
    url = base + node + parameters    
    responseDecode = getRequestUrl(url)   # [CODE 1] json 형식의 utf8로 인코딩된 문자열 형태다.
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode) # 응답 객체를 파이썬이 처리할 수 있는 json 형식으로 변환한다. 즉, json 파일을 딕셔너리로 변환한다. 왜냐면 나중에 items에 접근해야 하니깐 파이썬에서.

# [CODE 3]
def getPostData(post, jsonResult, cnt, lst, node):    
    title = post['title']
    if node == 'blog':
        for i in lst:
            if i in title:
                if i in diction:
                    diction[i] += 1
                else:
                    diction[i] = 1   
        jsonResult.append({'cnt':cnt, 'title':title})
    elif node == 'shop':
        cleaned_title = re.sub('<[^>]*>', '', title)
        jsonResult.append({
            'rank': cnt,
            'title': cleaned_title
        })

def getDictionary(node, lst=[]):
    srcText = input('검색어를 입력하세요: ') # 사용자 입력으로 받은 검색어 저장
    cnt = 0 # 검색 결과 카운트
    jsonResult = [] # 검색 결과를 정리하여 저장할 리스트 객체.
    # json파일은 약간 딕셔너리 형태임. key - value
    if node == 'blog':
        jsonResponse = getNaverSearch(node, srcText, 1, 100)  # [CODE 2] 네이버 뉴스 검색에 대한 응답을 저장하는 객체.
        while ((jsonResponse != None) and (jsonResponse['display'] != 0)): # display = 한 번에 가져온 결과 개수.  즉, display = len(items)이다.
            for post in jsonResponse['items']: # post = 응답받은 검색 결과 중 1개 저장한 객체 / items = 응답받은 검색 결과 전체. 내부 항목은 title, originallink,link,description, pubDate 등.
                cnt += 1                       # title = 기사 제목, originallink = 원래 언론사 사이트 주소, link = 네이버 뉴스 주소, description = 기사 요약, 본문 일부, pubDate = 기사 발행 시각
                getPostData(post, jsonResult, cnt, lst, node)  # [CODE 3]   # getPostData = 검색 결과 1개를 저장함.               
            start = jsonResponse['start'] + jsonResponse['display'] # 이제 다음 items들을 가져와야 하니깐 이렇게 함.
            if start == 1001: break    # 네이버 뉴스는 1000개까지만 무료 제공됨
            jsonResponse = getNaverSearch(node, srcText, start, 100)  #[CODE 2]       
    elif node == 'shop':
        jsonResponse = getNaverSearch(node, srcText, 1, 3)
        for post in jsonResponse['items']:
            getPostData(post, jsonResult, cnt, None, node)
    
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult,  indent = 4, sort_keys = True,  ensure_ascii = False) # 객체를 JSON 형식으로 변환.                    
        outfile.write(jsonFile)
        
    print("가져온 데이터 : %d 건" %(cnt))
    print ('%s_naver_%s.json SAVED' % (srcText, node))

    result = dict()

    if node == 'blog':
        top = sorted(diction.items(), key=lambda x: x[1], reverse=True)
        diction.clear()
        for i in range(3):
            result["top{}".format(i + 1)] = top[i][0]
    elif node == 'shop':
        for i in range(3):
            result["top{}".format(i + 1)] = jsonResult[i]['title']

    print(result)
    return (result, srcText)    
# [CODE 0]
def main():
    dict1, name1 = getDictionary("blog", lst_subin) # 검색어: 겨울노래
    dict2, name2 = getDictionary("blog", lst_seonggu) # 검색어 : 겨울음식
    dict3, name3 = getDictionary("blog", lst_taehoon) # 검색어 : 겨울여행
    dict4, name4 = getDictionary("shop") # 검색어 : 겨울
    df = pd.DataFrame({
        name1: dict1,
        name2: dict2,
        name3: dict3,
        name4: dict4
    })
    print(df)

if __name__ == '__main__':
    main()
    
# 위성 데이터 추출해서 강가만 표시하기.

가져온 데이터 : 1000 건
겨울노래_naver_blog.json SAVED
{'top1': '박효신', 'top2': '뷔', 'top3': '수지'}
가져온 데이터 : 1000 건
겨울음식_naver_blog.json SAVED
{'top1': '전골', 'top2': '국밥', 'top3': '만두'}
가져온 데이터 : 1000 건
겨울여행_naver_blog.json SAVED
{'top1': '제주', 'top2': '강원도', 'top3': '부산'}
가져온 데이터 : 0 건
겨울_naver_shop.json SAVED
{'top1': '이츠미 3가지기장 코듀로이 겨울 기모 하이웨스트 융 기모 안감 세미 와이드 여성 팬츠', 'top2': '나이키 남성 겨울 트레이닝복 세트 기모 트레이닝 바지 맨투맨 조거팬츠 츄리닝 한벌 셋업', 'top3': '오슬로 N59 남여공용 패딩 슬리퍼 방한 겨울 털 샌들 신발 경량 사무실'}
     겨울노래 겨울음식 겨울여행                                                겨울
top1  박효신   전골   제주   이츠미 3가지기장 코듀로이 겨울 기모 하이웨스트 융 기모 안감 세미 와이드 여성 팬츠
top2    뷔   국밥  강원도  나이키 남성 겨울 트레이닝복 세트 기모 트레이닝 바지 맨투맨 조거팬츠 츄리닝 한벌 셋업
top3   수지   만두   부산          오슬로 N59 남여공용 패딩 슬리퍼 방한 겨울 털 샌들 신발 경량 사무실


In [ ]:
client_id = '0XTfDtoDevGZuhx3BAck'
client_secret = '2Nxn8f6OdL'


# [CODE 1]
def getRequestUrl(url):    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200: # 400
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
        else:
            return None
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

# [CODE 2]
def getNaverSearch(node, srcText, start, display):    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display) # .parse.quote는 이제 srcText를 인코딩 해 넣는다. 한글은 깨지기 때문에.
    
    url = base + node + parameters    
    responseDecode = getRequestUrl(url)   # [CODE 1] json 형식의 utf8로 인코딩된 문자열 형태다.
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode) # 응답 객체를 파이썬이 처리할 수 있는 json 형식으로 변환한다. 즉, json 파일을 딕셔너리로 변환한다. 왜냐면 나중에 items에 접근해야 하니깐 파이썬에서.

# [CODE 3]
def getPostData(post, jsonResult, cnt):    
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    
    pDate = datetime.datetime.strptime(post['pubDate'],  '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 
'org_link':org_link,   'link': org_link,   'pDate':pDate})
        

# [CODE 0]
def main():
    node = 'news'   # 크롤링 대상 노드 : 네이버 '뉴스' # 네이버 검색 API에서 검색할 대상 노드
    srcText = input('검색어를 입력하세요: ') # 사용자 입력으로 받은 검색어 저장
    cnt = 0 # 검색 결과 카운트
    jsonResult = [] # 검색 결과를 정리하여 저장할 리스트 객체.
    # json파일은 약간 딕셔너리 형태임. key - value
    jsonResponse = getNaverSearch(node, srcText, 1, 100)  # [CODE 2] 네이버 뉴스 검색에 대한 응답을 저장하는 객체.
    
    total = jsonResponse['total'] # 전체 검색 결과 개수
    # jsonFile: JSON파일에 저장할 데이터를 담은 객체.
    # total = 350   한 번에 items를 100개가. 101 200
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)): # display = 한 번에 가져온 결과 개수.  즉, display = len(items)이다.
        for post in jsonResponse['items']: # post = 응답받은 검색 결과 중 1개 저장한 객체 / items = 응답받은 검색 결과 전체. 내부 항목은 title, originallink,link,description, pubDate 등.
            cnt += 1                       # title = 기사 제목, originallink = 원래 언론사 사이트 주소, link = 네이버 뉴스 주소, description = 기사 요약, 본문 일부, pubDate = 기사 발행 시각
            getPostData(post, jsonResult, cnt)  # [CODE 3]   # getPostData = 검색 결과 1개를 저장함.     
        
        start = jsonResponse['start'] + jsonResponse['display'] # 이제 다음 items들을 가져와야 하니깐 이렇게 함.
        if start == 1001: break    # 네이버 뉴스는 1000개까지만 무료 제공됨
        jsonResponse = getNaverSearch(node, srcText, start, 100)  #[CODE 2]       

    print('전체 검색 : %d 건' %total)
    
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult,  indent = 4, sort_keys = True,  ensure_ascii = False) # 객체를 JSON 형식으로 변환.
                        
        outfile.write(jsonFile)
        
    print("가져온 데이터 : %d 건" %(cnt))
    print ('%s_naver_%s.json SAVED' % (srcText, node))
    
if __name__ == '__main__':
    main()


[2025-11-24 12:04:33.536006] Url Request Success
[2025-11-24 12:04:33.649037] Url Request Success
[2025-11-24 12:04:33.767737] Url Request Success
[2025-11-24 12:04:33.889162] Url Request Success
[2025-11-24 12:04:34.034067] Url Request Success
[2025-11-24 12:04:34.173881] Url Request Success
[2025-11-24 12:04:34.306687] Url Request Success
[2025-11-24 12:04:34.469461] Url Request Success
[2025-11-24 12:04:34.622071] Url Request Success
[2025-11-24 12:04:34.772102] Url Request Success
전체 검색 : 265596 건
가져온 데이터 : 1000 건
로또_naver_news.json SAVED


In [ ]:
import os
import sys
import urllib.request
import datetime
import time
import json

### 최신 뉴스만 모으기

In [6]:
client_id = '0XTfDtoDevGZuhx3BAck'
client_secret = '2Nxn8f6OdL'


# [CODE 1]
def getRequestUrl(url):    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200: # 400
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
        else:
            return None
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

# [CODE 2]
def getNaverSearch(node, srcText, start, display):    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display) # .parse.quote는 이제 srcText를 인코딩 해 넣는다. 한글은 깨지기 때문에.
    
    url = base + node + parameters    
    responseDecode = getRequestUrl(url)   # [CODE 1] json 형식의 utf8로 인코딩된 문자열 형태다.
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode) # 응답 객체를 파이썬이 처리할 수 있는 json 형식으로 변환한다. 즉, json 파일을 딕셔너리로 변환한다. 왜냐면 나중에 items에 접근해야 하니깐 파이썬에서.

# [CODE 3]
def getPostData(post, jsonResult, cnt):    
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    
    pDate = datetime.datetime.strptime(post['pubDate'],  '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 
'org_link':org_link,   'link': org_link,   'pDate':pDate})
        

# [CODE 0]
def main():
    node = 'news'   # 크롤링 대상 노드 : 네이버 '뉴스' # 네이버 검색 API에서 검색할 대상 노드
    srcText = input('검색어를 입력하세요: ') # 사용자 입력으로 받은 검색어 저장
    cnt = 0 # 검색 결과 카운트
    jsonResult = [] # 검색 결과를 정리하여 저장할 리스트 객체.
    # json파일은 약간 딕셔너리 형태임. key - value
    jsonResponse = getNaverSearch(node, srcText, 1, 100)  # [CODE 2] 네이버 뉴스 검색에 대한 응답을 저장하는 객체.
    
    total = jsonResponse['total'] # 전체 검색 결과 개수
    # jsonFile: JSON파일에 저장할 데이터를 담은 객체.
    # total = 350   한 번에 items를 100개가. 101 200
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)): # display = 한 번에 가져온 결과 개수.  즉, display = len(items)이다.
        for post in jsonResponse['items']: # post = 응답받은 검색 결과 중 1개 저장한 객체 / items = 응답받은 검색 결과 전체. 내부 항목은 title, originallink,link,description, pubDate 등.
            date = datetime.datetime.strptime(post['pubDate'],  '%a, %d %b %Y %H:%M:%S +0900')
            if date.date() == datetime.datetime.now().date():
                cnt += 1                       # title = 기사 제목, originallink = 원래 언론사 사이트 주소, link = 네이버 뉴스 주소, description = 기사 요약, 본문 일부, pubDate = 기사 발행 시각
                getPostData(post, jsonResult, cnt)  # [CODE 3]   # getPostData = 검색 결과 1개를 저장함.     
            
        start = jsonResponse['start'] + jsonResponse['display'] # 이제 다음 items들을 가져와야 하니깐 이렇게 함.
        if start == 1001: break    # 네이버 뉴스는 1000개까지만 무료 제공됨
        jsonResponse = getNaverSearch(node, srcText, start, 100)  #[CODE 2]       

    print('전체 검색 : %d 건' %total)
    
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult,  indent = 4, sort_keys = True,  ensure_ascii = False) # 객체를 JSON 형식으로 변환.
                        
        outfile.write(jsonFile)
        
    print("가져온 데이터 : %d 건" %(cnt))
    print ('%s_naver_%s.json SAVED' % (srcText, node))
    
if __name__ == '__main__':
    main()


[2025-11-24 14:25:35.531556] Url Request Success
[2025-11-24 14:25:35.645093] Url Request Success
[2025-11-24 14:25:35.746936] Url Request Success
[2025-11-24 14:25:35.873498] Url Request Success
[2025-11-24 14:25:36.004187] Url Request Success
[2025-11-24 14:25:36.133764] Url Request Success
[2025-11-24 14:25:36.281822] Url Request Success
[2025-11-24 14:25:36.419372] Url Request Success
[2025-11-24 14:25:36.565479] Url Request Success
[2025-11-24 14:25:36.732159] Url Request Success
전체 검색 : 40237 건
가져온 데이터 : 46 건
아일릿_naver_news.json SAVED


### 특정 키워드로 300개 크롤링하고 언론사별 개수 집계하기

In [24]:
import re

client_id = '0XTfDtoDevGZuhx3BAck'
client_secret = '2Nxn8f6OdL'

singer = dict()
music = dict()

# [CODE 1]
def getRequestUrl(url):    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200: # 400
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
        else:
            return None
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

# [CODE 2]
def getNaverSearch(node, srcText, start, display):    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display) # .parse.quote는 이제 srcText를 인코딩 해 넣는다. 한글은 깨지기 때문에.
    
    url = base + node + parameters    
    responseDecode = getRequestUrl(url)   # [CODE 1] json 형식의 utf8로 인코딩된 문자열 형태다.
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode) # 응답 객체를 파이썬이 처리할 수 있는 json 형식으로 변환한다. 즉, json 파일을 딕셔너리로 변환한다. 왜냐면 나중에 items에 접근해야 하니깐 파이썬에서.

# [CODE 3]
def getPostData(post, jsonResult, cnt):    
    
    title = post['title']
    length = len(title)
    title = title.split(' ')

    if '-' in title:
        idx = title.index('-')
    else:
        idx = -1

    if idx != -1 and idx != 0 and (idx + 1 != length):
        if title[idx - 1] in singer:
            singer[title[idx - 1]] += 1
        else:
            singer[title[idx - 1]] = 1
        if title[idx + 1] in music:
            music[title[idx + 1]] += 1
        else:
            music[title[idx + 1]] = 1

    description = post['description']
    bloggername = post['bloggername']
    link = post['bloggerlink']
    
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 
'bloggername':bloggername,   'link': link})
        

# [CODE 0]
def main():
    node = 'blog'   # 크롤링 대상 노드 : 네이버 '뉴스' # 네이버 검색 API에서 검색할 대상 노드
    srcText = input('검색어를 입력하세요: ') # 사용자 입력으로 받은 검색어 저장
    cnt = 0 # 검색 결과 카운트
    jsonResult = [] # 검색 결과를 정리하여 저장할 리스트 객체.
    # json파일은 약간 딕셔너리 형태임. key - value
    jsonResponse = getNaverSearch(node, srcText, 1, 100)  # [CODE 2] 네이버 뉴스 검색에 대한 응답을 저장하는 객체.
    
    total = jsonResponse['total'] # 전체 검색 결과 개수
    # jsonFile: JSON파일에 저장할 데이터를 담은 객체.
    # total = 350   한 번에 items를 100개가. 101 200
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)): # display = 한 번에 가져온 결과 개수.  즉, display = len(items)이다.
        for post in jsonResponse['items']: # post = 응답받은 검색 결과 중 1개 저장한 객체 / items = 응답받은 검색 결과 전체. 내부 항목은 title, originallink,link,description, pubDate 등.
            cnt += 1                       # title = 기사 제목, originallink = 원래 언론사 사이트 주소, link = 네이버 뉴스 주소, description = 기사 요약, 본문 일부, pubDate = 기사 발행 시각
            getPostData(post, jsonResult, cnt)  # [CODE 3]   # getPostData = 검색 결과 1개를 저장함.     
            
        start = jsonResponse['start'] + jsonResponse['display'] # 이제 다음 items들을 가져와야 하니깐 이렇게 함.
        if start == 1001: break    # 네이버 뉴스는 1000개까지만 무료 제공됨
        jsonResponse = getNaverSearch(node, srcText, start, 100)  #[CODE 2]       

    print('전체 검색 : %d 건' %total)
    
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult,  indent = 4, sort_keys = True,  ensure_ascii = False) # 객체를 JSON 형식으로 변환.
                        
        outfile.write(jsonFile)
        
    print("가져온 데이터 : %d 건" %(cnt))
    print ('%s_naver_%s.json SAVED' % (srcText, node))

    print("Singer dictionary:", singer)
    print("Music dictionary:", music)
    
if __name__ == '__main__':
    main()
    


[2025-11-24 15:57:49.732231] Url Request Success
[2025-11-24 15:57:49.809089] Url Request Success
[2025-11-24 15:57:49.988491] Url Request Success
[2025-11-24 15:57:50.062990] Url Request Success
[2025-11-24 15:57:50.140110] Url Request Success
[2025-11-24 15:57:50.338286] Url Request Success
[2025-11-24 15:57:50.410126] Url Request Success
[2025-11-24 15:57:50.607741] Url Request Success
[2025-11-24 15:57:50.672900] Url Request Success
[2025-11-24 15:57:50.755562] Url Request Success
전체 검색 : 34958 건
가져온 데이터 : 1000 건
겨울 노래_naver_blog.json SAVED
Singer dictionary: {'EXO': 1, '(존박)': 1, 'sia': 1, 'Sparks': 1, '스텔라장': 1, 'SEINY': 1, 'TOP10': 1, '뉘뉘(NUITNUIT)': 1, '빅뱅': 1, '(찬)': 1, '(hartts)': 1, '유리상자': 1, '3편': 1, '오세영': 3, 'Tenors': 1, '예빛': 1, 'Girl': 1, '설경': 1, '추천': 3, '공상': 1, 'City)': 1, '디바': 1, '(NavyQuokka)': 1, 'Jiwoo': 1, '노래</b>': 6, '(2)': 1, '(피엘)': 1, 'Blue': 1, '<b>겨울노래</b>': 7, '새벽공방': 1, 'green': 1, '(Beaver)': 1, 'Snowman': 1, 'DeVita': 1, '추천❄': 1, '(1)': 1, '(쏠)': 

In [25]:
import re

client_id = '0XTfDtoDevGZuhx3BAck'
client_secret = '2Nxn8f6OdL'

singer = dict()
music = dict()

# [CODE 1]
def getRequestUrl(url):    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200: # 400
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
        else:
            return None
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

# [CODE 2]
def getNaverSearch(node, srcText, start, display):    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display) # .parse.quote는 이제 srcText를 인코딩 해 넣는다. 한글은 깨지기 때문에.
    
    url = base + node + parameters    
    responseDecode = getRequestUrl(url)   # [CODE 1] json 형식의 utf8로 인코딩된 문자열 형태다.
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode) # 응답 객체를 파이썬이 처리할 수 있는 json 형식으로 변환한다. 즉, json 파일을 딕셔너리로 변환한다. 왜냐면 나중에 items에 접근해야 하니깐 파이썬에서.

# [CODE 3]
def getPostData(post, jsonResult, cnt):    
    
    title = post['title']
    description = post['description']
    bloggername = post['bloggername']
    link = post['bloggerlink']
    
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 
'bloggername':bloggername,   'link': link})
        

# [CODE 0]
def main():
    node = 'blog'   # 크롤링 대상 노드 : 네이버 '뉴스' # 네이버 검색 API에서 검색할 대상 노드
    srcText = input('검색어를 입력하세요: ') # 사용자 입력으로 받은 검색어 저장
    cnt = 0 # 검색 결과 카운트
    jsonResult = [] # 검색 결과를 정리하여 저장할 리스트 객체.
    # json파일은 약간 딕셔너리 형태임. key - value
    jsonResponse = getNaverSearch(node, srcText, 1, 100)  # [CODE 2] 네이버 뉴스 검색에 대한 응답을 저장하는 객체.
    
    total = jsonResponse['total'] # 전체 검색 결과 개수
    # jsonFile: JSON파일에 저장할 데이터를 담은 객체.
    # total = 350   한 번에 items를 100개가. 101 200
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)): # display = 한 번에 가져온 결과 개수.  즉, display = len(items)이다.
        for post in jsonResponse['items']: # post = 응답받은 검색 결과 중 1개 저장한 객체 / items = 응답받은 검색 결과 전체. 내부 항목은 title, originallink,link,description, pubDate 등.
            cnt += 1                       # title = 기사 제목, originallink = 원래 언론사 사이트 주소, link = 네이버 뉴스 주소, description = 기사 요약, 본문 일부, pubDate = 기사 발행 시각
            getPostData(post, jsonResult, cnt)  # [CODE 3]   # getPostData = 검색 결과 1개를 저장함.     
            
        start = jsonResponse['start'] + jsonResponse['display'] # 이제 다음 items들을 가져와야 하니깐 이렇게 함.
        if start == 1001: break    # 네이버 뉴스는 1000개까지만 무료 제공됨
        jsonResponse = getNaverSearch(node, srcText, start, 100)  #[CODE 2]       

    print('전체 검색 : %d 건' %total)
    
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult,  indent = 4, sort_keys = True,  ensure_ascii = False) # 객체를 JSON 형식으로 변환.
                        
        outfile.write(jsonFile)
        
    print("가져온 데이터 : %d 건" %(cnt))
    print ('%s_naver_%s.json SAVED' % (srcText, node))

    print("Singer dictionary:", singer)
    print("Music dictionary:", music)
    
if __name__ == '__main__':
    main()
    


[2025-11-24 15:59:33.252250] Url Request Success
[2025-11-24 15:59:33.429943] Url Request Success
[2025-11-24 15:59:33.508281] Url Request Success
[2025-11-24 15:59:33.686872] Url Request Success
[2025-11-24 15:59:33.868744] Url Request Success
[2025-11-24 15:59:33.942560] Url Request Success
[2025-11-24 15:59:34.138957] Url Request Success
[2025-11-24 15:59:34.210080] Url Request Success
[2025-11-24 15:59:34.397622] Url Request Success
[2025-11-24 15:59:34.471394] Url Request Success
전체 검색 : 34958 건
가져온 데이터 : 1000 건
겨울 노래_naver_blog.json SAVED
Singer dictionary: {}
Music dictionary: {}


In [45]:
client_id = '0XTfDtoDevGZuhx3BAck'
client_secret = '2Nxn8f6OdL'

lst = ["EXO", "존박", "수지", "sia","Jordin Sparks", "규현", "스텔라장", "뷔", "BTS", "박효신", "보이넥스트도어", "SEINY", "써니힐", "임영웅", "강수지", "빅뱅", "조용필"]
singer = dict()


# [CODE 1]
def getRequestUrl(url):    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200: # 400
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
        else:
            return None
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

# [CODE 2]
def getNaverSearch(node, srcText, start, display):    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display) # .parse.quote는 이제 srcText를 인코딩 해 넣는다. 한글은 깨지기 때문에.
    
    url = base + node + parameters    
    responseDecode = getRequestUrl(url)   # [CODE 1] json 형식의 utf8로 인코딩된 문자열 형태다.
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode) # 응답 객체를 파이썬이 처리할 수 있는 json 형식으로 변환한다. 즉, json 파일을 딕셔너리로 변환한다. 왜냐면 나중에 items에 접근해야 하니깐 파이썬에서.

# [CODE 3]
def getPostData(post, jsonResult, cnt):    
    
    title = post['title']
    for i in lst:
        if i in title:
            if i in singer:
                singer[i] += 1
            else:
                singer[i] = 1

    description = post['description']
    bloggername = post['bloggername']
    link = post['bloggerlink']
    
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 'bloggername':bloggername,   'link': link})
        

# [CODE 0]
def main():
    node = 'blog'   # 크롤링 대상 노드 : 네이버 '뉴스' # 네이버 검색 API에서 검색할 대상 노드
    srcText = input('검색어를 입력하세요: ') # 사용자 입력으로 받은 검색어 저장
    cnt = 0 # 검색 결과 카운트
    jsonResult = [] # 검색 결과를 정리하여 저장할 리스트 객체.
    # json파일은 약간 딕셔너리 형태임. key - value
    jsonResponse = getNaverSearch(node, srcText, 1, 100)  # [CODE 2] 네이버 뉴스 검색에 대한 응답을 저장하는 객체.
    
    total = jsonResponse['total'] # 전체 검색 결과 개수
    # jsonFile: JSON파일에 저장할 데이터를 담은 객체.
    # total = 350   한 번에 items를 100개가. 101 200
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)): # display = 한 번에 가져온 결과 개수.  즉, display = len(items)이다.
        for post in jsonResponse['items']: # post = 응답받은 검색 결과 중 1개 저장한 객체 / items = 응답받은 검색 결과 전체. 내부 항목은 title, originallink,link,description, pubDate 등.
            cnt += 1                       # title = 기사 제목, originallink = 원래 언론사 사이트 주소, link = 네이버 뉴스 주소, description = 기사 요약, 본문 일부, pubDate = 기사 발행 시각
            getPostData(post, jsonResult, cnt)  # [CODE 3]   # getPostData = 검색 결과 1개를 저장함.     
            
        start = jsonResponse['start'] + jsonResponse['display'] # 이제 다음 items들을 가져와야 하니깐 이렇게 함.
        if start == 1001: break    # 네이버 뉴스는 1000개까지만 무료 제공됨
        jsonResponse = getNaverSearch(node, srcText, start, 100)  #[CODE 2]       

    print('전체 검색 : %d 건' %total)
    
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult,  indent = 4, sort_keys = True,  ensure_ascii = False) # 객체를 JSON 형식으로 변환.
                        
        outfile.write(jsonFile)
        
    print("가져온 데이터 : %d 건" %(cnt))
    print ('%s_naver_%s.json SAVED' % (srcText, node))

    top = sorted(singer.items(), key=lambda x: x[1], reverse=True)
    result = dict()
    for i in range(3):
        result["top{}".format(i + 1)] = top[i][0]

    print(result)

if __name__ == '__main__':
    main()
    
# 위성 데이터 추출해서 강가만 표시하기.

[2025-11-24 17:36:13.803213] Url Request Success
[2025-11-24 17:36:13.886172] Url Request Success
[2025-11-24 17:36:14.077913] Url Request Success
[2025-11-24 17:36:14.160913] Url Request Success
[2025-11-24 17:36:14.268844] Url Request Success
[2025-11-24 17:36:14.338030] Url Request Success
[2025-11-24 17:36:14.409303] Url Request Success
[2025-11-24 17:36:14.474346] Url Request Success
[2025-11-24 17:36:14.661584] Url Request Success
[2025-11-24 17:36:14.738263] Url Request Success
전체 검색 : 34965 건
가져온 데이터 : 1000 건
겨울 노래_naver_blog.json SAVED
{'top1': '박효신', 'top2': '뷔', 'top3': '수지'}


In [ ]:
client_id = '0XTfDtoDevGZuhx3BAck'
client_secret = '2Nxn8f6OdL'

lst = ["EXO", "존박", "수지", "sia","Jordin Sparks", "규현", "스텔라장", "뷔", "BTS", "박효신", "보이넥스트도어", "SEINY", "써니힐", "임영웅", "강수지", "빅뱅", "조용필"]
singer = dict()


# [CODE 1]
def getRequestUrl(url):    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200: # 400
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
        else:
            return None
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

# [CODE 2]
def getNaverSearch(node, srcText, start, display):    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display) # .parse.quote는 이제 srcText를 인코딩 해 넣는다. 한글은 깨지기 때문에.
    
    url = base + node + parameters    
    responseDecode = getRequestUrl(url)   # [CODE 1] json 형식의 utf8로 인코딩된 문자열 형태다.
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode) # 응답 객체를 파이썬이 처리할 수 있는 json 형식으로 변환한다. 즉, json 파일을 딕셔너리로 변환한다. 왜냐면 나중에 items에 접근해야 하니깐 파이썬에서.

# [CODE 3]
def getPostData(post, jsonResult, cnt, lst):    
    
    title = post['title']
    for i in lst:
        if i in title:
            if i in singer:
                singer[i] += 1
            else:
                singer[i] = 1

    description = post['description']
    bloggername = post['bloggername']
    link = post['bloggerlink']
    
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 'bloggername':bloggername,   'link': link})
        

def getDictionary(node, lst):
    #node = 'blog'   # 크롤링 대상 노드 : 네이버 '뉴스' # 네이버 검색 API에서 검색할 대상 노드
    srcText = input('검색어를 입력하세요: ') # 사용자 입력으로 받은 검색어 저장
    cnt = 0 # 검색 결과 카운트
    jsonResult = [] # 검색 결과를 정리하여 저장할 리스트 객체.
    # json파일은 약간 딕셔너리 형태임. key - value
    jsonResponse = getNaverSearch(node, srcText, 1, 100)  # [CODE 2] 네이버 뉴스 검색에 대한 응답을 저장하는 객체.
    
    total = jsonResponse['total'] # 전체 검색 결과 개수
    # jsonFile: JSON파일에 저장할 데이터를 담은 객체.
    # total = 350   한 번에 items를 100개가. 101 200
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)): # display = 한 번에 가져온 결과 개수.  즉, display = len(items)이다.
        for post in jsonResponse['items']: # post = 응답받은 검색 결과 중 1개 저장한 객체 / items = 응답받은 검색 결과 전체. 내부 항목은 title, originallink,link,description, pubDate 등.
            cnt += 1                       # title = 기사 제목, originallink = 원래 언론사 사이트 주소, link = 네이버 뉴스 주소, description = 기사 요약, 본문 일부, pubDate = 기사 발행 시각
            getPostData(post, jsonResult, cnt, lst)  # [CODE 3]   # getPostData = 검색 결과 1개를 저장함.     
            
        start = jsonResponse['start'] + jsonResponse['display'] # 이제 다음 items들을 가져와야 하니깐 이렇게 함.
        if start == 1001: break    # 네이버 뉴스는 1000개까지만 무료 제공됨
        jsonResponse = getNaverSearch(node, srcText, start, 100)  #[CODE 2]       

    print('전체 검색 : %d 건' %total)
    
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult,  indent = 4, sort_keys = True,  ensure_ascii = False) # 객체를 JSON 형식으로 변환.
                        
        outfile.write(jsonFile)
        
    print("가져온 데이터 : %d 건" %(cnt))
    print ('%s_naver_%s.json SAVED' % (srcText, node))

    top = sorted(singer.items(), key=lambda x: x[1], reverse=True)
    result = dict()
    for i in range(3):
        result["top{}".format(i + 1)] = top[i][0]

    print(result)
    return result
    
# [CODE 0]
def main():
    dict1 = getDictionary("blog", lst)

if __name__ == '__main__':
    main()
    
# 위성 데이터 추출해서 강가만 표시하기.

[2025-11-24 17:47:36.062171] Url Request Success
[2025-11-24 17:47:36.138933] Url Request Success
[2025-11-24 17:47:36.316168] Url Request Success
[2025-11-24 17:47:36.506626] Url Request Success
[2025-11-24 17:47:36.688899] Url Request Success
[2025-11-24 17:47:36.764854] Url Request Success
[2025-11-24 17:47:36.944992] Url Request Success
[2025-11-24 17:47:37.148181] Url Request Success
[2025-11-24 17:47:37.338086] Url Request Success
[2025-11-24 17:47:37.549528] Url Request Success
전체 검색 : 34967 건
가져온 데이터 : 1000 건
겨울 노래_naver_blog.json SAVED
{'top1': '박효신', 'top2': '뷔', 'top3': '수지'}
